## Mini Project 2

## Lecture Transcript Word Frequency Analysis

## 100 Points.

```mermaid
graph TB
    
EPT["1.1<br>extract_pdf_text"]
RH["1.2<br>remove_headers"]
RP["1.3<br>remove_parentheses"]
RS["1.4<br>remove_speakers"]

SIS["2.1<br>split_into_sentences"]
TS["2.2<br>tokenize_sentence"]
RSW["2.3<br>remove_stopwords"]

SAC["3.1<br>save_as_csv"]

CWF["4.1<br>update_word_frequency"]
PWFD["4.2<br>plot_word_frequency_dist"]
SWBF["4.3<br>sort_words_by_frequency"]
FBIOW["4.4<br>find_bucket_index_of_words"]
SAJ["4.5<br>save_as_json"]
PSWF["4.6<br>plot_stacked_word_frequency"]

EPT --> RH
RH --> RP
RP --> RS

RS --> SIS
SIS --> TS
TS --> RSW

RSW --> SAC

RSW --> CWF
CWF --> PWFD
CWF --> SWBF
SWBF --> FBIOW

FBIOW --> SAJ

FBIOW --> PSWF
CWF --> PSWF

subgraph "Extract Text Contents"
    EPT
    RH
    RP
    RS
end

subgraph "Natural Language Preprocessing"
    SIS
    TS
    RSW
end

subgraph "Save the Cleaned Text"
    SAC
end

subgraph "Word Frequency Analysis"
    CWF
    PWFD
    SWBF
    FBIOW
    SAJ
    PSWF
end
```

### Task 1 (30 pts): Extract text contents

#### 1.1 (5 pts): Extract the text from a PDF file.

Write a function `extract_pdf_text(file_path)` that:

- **Input:** A string `file_path` specifying the path to a PDF file.
- **Output:** A string containing all the text extracted from the PDF file.

**Note:**
- For this task, you may use the [`pypdf` module](https://github.com/py-pdf/pypdf) to extract text from the PDF file.
  - To install the module, open a terminal and run 
  ```bash
  pip install pypdf
  ```
  If you are using Linux or MacOS, you may need to run `pip3` instead of `pip`.
- Join the text from different pages with a space.

In [11]:
from pypdf import PdfReader

def extract_pdf_text(file_path):
    """
    Extracts text from a PDF file as a string
    IN: file_path, str, path to the PDF file
    OUT: text, str, extracted text from all pages joined together with whitespaces
    """
    # create a reader to read the PDF file
    reader = PdfReader(file_path)

    text_list = [] # list to collect text from each page
    for page in reader.pages:
        # extract text from the page
        page_text = page.extract_text() 
        # append the text to the list
        text_list.append(page_text)

    # join the text from all pages with whitespaces
    text = ' '.join(text_list)
    return text

#### 1.2 (7 pts): Remove headers.

Write a function `remove_headers(text)` that:

- **Input:** A string `text` extracted from the PDF file.
- **Output:** A string that does not contain the header texts.

**Note:**
- Some lecture transcripts may contain a header with the service name "StreamBox" on the first page. Remove it if it is present. See `transcript 1.pdf` for an example.
- Some lecture transcripts may start with a Disclaimer. Remove all the text enclosed by `**********DISCLAIMER**********` (included). See `transcript 4.pdf` for an example.


In [14]:
def remove_headers(text):
    """
    Removes headers from the text
    IN: text, str, text to remove headers from
    OUT: text, str, text with headers removed
    """
    # if there is any leading "StreamBox", remove it
    if text[:9] == 'StreamBox':
        text = text.replace("StreamBox","",1)

    # if there is any **********DISCLAIMER**********, find the matched end, and remove it
    firstI = text.find("**********DISCLAIMER**********")
    if(firstI != -1):
        secondI = text.find("**********DISCLAIMER**********",firstI)
        text = text[secondI + len("**********DISCLAIMER**********"):]
    return text


"  \nTHE FOLLOWING IS AN UNEDITED DRAFT TRANSLATION. THIS TRANSCRIPT MAY NOT BE \nVERBATIM, HAS NOT BEEN PROOFREAD AND MAY CONTAIN ERRORS. THESE CHECK WITH \nTHE SPEAKERS FOR ANY CLARIFICATION.  \nTHIS TRANSCRIPT MAY NOT BE COPIED OR DISSEMINATED UNLESS YOU OBTAIN WRITTEN \nPERMISSION FROM THE OFFICE OR SERVICE DEPARTMENT THAT IS PROVIDING CART \nCAPTIONING TO YOU.  \nTHIS TRANSCRIPT MAY NOT USED IN A COURT OF LAW. -FT  \n**********DISCLAIMER**********  \nRNB ZR CS 198 8:30-9:50 a.m. ET  \nSeptember 19, 2024  \n>> INSTRUCTOR: For sets, we also use parentheses. And there's an important distinction \nbetween these three types of data subsets and this one . Very important distinction. The \ndictionaries are (Indiscernible) it's quite different to this. We call them characteristic . This \ntwo. There are sequences. These two. Sets -- let me do here -- sets are different from these \ntwo in that sense. Sets have no sequences . When we say that something is a sequence, it's \nbecause it exis

#### 1.3 (10 pts): Remove transcription system notes enclosed by parentheses.

Write a function `remove_parentheses(text)` that:

- **Input:** A string `text` that has been processed by the previous functions.
- **Output:** A string that does not contain the transcription system notes enclosed by parentheses.

**Note:**
- Some lecture transcripts use parentheses to provide additional information about the transcription system. Remove all the text enclosed by parentheses. For example, `[Captioner standing by]` in `transcript 1.pdf`.
- There are NO nested parentheses in the lecture transcripts.
- The parentheses may either be round parentheses `()` or square parentheses `[]`.
- Some parentheses may be followed immediately by a period. For example, `(Away From Mic).` in `transcript 4.pdf`. In this case, you should remove the period as well.
- You are welcome to use regular expressions to solve this task, but it is not required. You can use Python built-in string methods to achieve the same result.


In [33]:
import re # if you don't know regular expressions, ignore this import, and use Python plain vanilla string methods

def remove_parentheses(text):
    """
    Removes text within parentheses
    IN: text, str, text to remove notes enclosed in parentheses
    OUT: text, str, text without parentheses notes
    """
    # remove round parentheses () and the text within them. If there is a trailing period, remove it as well
    while(text.find('(') != -1):
        index = text.find('(')
        sindex = text.find(')',index)
        if(text[sindex+1] == '.'):
            sindex = sindex +1
        text = text[:index-1] + text[sindex +1:]

    # remove square parentheses [] and the text within them. If there is a trailing period, remove it as well
    while(text.find('[') != -1):
        index = text.find('[')
        sindex = text.find(']',index)
        if(text[sindex+1] == '.'):
            sindex = sindex +1
        text = text[:index-1] + text[sindex+ 1:]

    return text

"StreamBox \n**********DISCLAIMER**********  \nTHE FOLLOWING IS AN UNEDITED DRAFT TRANSLATION. THIS TRANSCRIPT MAY NOT BE \nVERBATIM, HAS NOT BEEN PROOFREAD AND MAY CONTAIN ERRORS. THESE CHECK WITH \nTHE SPEAKERS FOR ANY CLARIFICATION.  \nTHIS TRANSCRIPT MAY NOT BE COPIED OR DISSEMINATED UNLESS YOU OBTAIN WRITTEN \nPERMISSION FROM THE OFFICE OR SERVICE DEPARTMENT THAT IS PROVIDING CART \nCAPTIONING TO YOU.  \nTHIS TRANSCRIPT MAY NOT USED IN A COURT OF LAW. -FT  \n**********DISCLAIMER**********  \nRNB ZR CS 198 8:30-9:50 a.m. ET  \nSeptember 19, 2024  \n>> INSTRUCTOR: For sets, we also use parentheses. And there's an important distinction \nbetween these three types of data subsets and this one . Very important distinction. The \ndictionaries are it's quite different to this. We call them characteristic . This \ntwo. There are sequences. These two. Sets -- let me do here -- sets are different from these \ntwo in that sense. Sets have no sequences . When we say that something is a sequen

#### 1.4 (8 pts): Remove speaker notations.

Write a function `remove_speakers(text)` that:

- **Input:** A string `text` that has been processed by the previous functions.
- **Output:** A string that does not contain the speaker notations.

**Note:**
- Speaker notations start with two greater-than symbols `>>`, followed by the speaker's name in uppercase letters, and end with a colon `:`. For example, `>> INSTRUCTOR:`.
- `>>` symbols are used only for speaker notations and NOT for any other purposes.
- You are welcome to use regular expressions to solve this task, but it is not required. You can use Python built-in string methods to achieve the same result.

In [12]:
import re # if you don't know regular expressions, ignore this import, and use Python plain vanilla string methods

def remove_speakers(text):
    """
    Removes speaker names from the text
    IN: text, str, text to remove speaker notations from
    OUT: text, str, text without speaker notations
    """
    # remove speaker notations
    text = re.sub(r"\s*>> \S+:","",text)

    return text


"StreamBox \n**********DISCLAIMER**********  \nTHE FOLLOWING IS AN UNEDITED DRAFT TRANSLATION. THIS TRANSCRIPT MAY NOT BE \nVERBATIM, HAS NOT BEEN PROOFREAD AND MAY CONTAIN ERRORS. THESE CHECK WITH \nTHE SPEAKERS FOR ANY CLARIFICATION.  \nTHIS TRANSCRIPT MAY NOT BE COPIED OR DISSEMINATED UNLESS YOU OBTAIN WRITTEN \nPERMISSION FROM THE OFFICE OR SERVICE DEPARTMENT THAT IS PROVIDING CART \nCAPTIONING TO YOU.  \nTHIS TRANSCRIPT MAY NOT USED IN A COURT OF LAW. -FT  \n**********DISCLAIMER**********  \nRNB ZR CS 198 8:30-9:50 a.m. ET  \nSeptember 19, 2024 For sets, we also use parentheses. And there's an important distinction \nbetween these three types of data subsets and this one . Very important distinction. The \ndictionaries are (Indiscernible) it's quite different to this. We call them characteristic . This \ntwo. There are sequences. These two. Sets -- let me do here -- sets are different from these \ntwo in that sense. Sets have no sequences . When we say that something is a sequence

### Task 2 (20 pts): Natural Language Preprocessing

In this task, you may use a Natural Language Processing module [`spaCy`](https://spacy.io/) to preprocess the text data.

**Installation Instructions:**
1. To install the spaCy module, open a terminal and run:
   ```bash
   pip install spacy
   python -m spacy download en_core_web_sm
    ```
    If you are using Linux or MacOS, you may need to run `pip3` instead of `pip`, and `python3` instead of `python`.

#### 2.1 (6 pts): Split text into sentences.

Write a function `split_into_sentences(text)` that

- **Input:** A string `text` that has been processed by the previous functions.
- **Output:** A list of strings, where each string is a sentence.

**Note:**

- `spaCy` provides a sentence segmentation method that can be used to split the text into sentences. Below is an example of how to implement this functionality:

  ```python
  import spacy

  nlp = spacy.load("en_core_web_sm") # Load the English language model

  text = "This is a sentence. This is another sentence."
  doc = nlp(text) # Process the input text

  # Extract sentences from the processed document
  sentences = [sent.text for sent in doc.sents]
  print(sentences)
  ```
  The printed output will be 
  ```python
  ['This is a sentence.', 'This is another sentence.']
  ```

In [127]:
import spacy

nlp = spacy.load("en_core_web_sm")

def split_into_sentences(text):
    """
    Splits text into sentences
    IN: text, str, text to split into sentences
    OUT: sentences, list[str], sentences in the text
    """
    # join lines into a single paragraph
    text = text.replace("\n", " ")

    # remove leading and trailing whitespaces in the text
    """__Your_Code_Here__"""

    # use spaCy to split the text into sentences
    """__Your_Code_Here__"""

    return sentences

#### 2.2 (8 pts): Tokenize sentences and remove punctuations and spaces.

Write a function `tokenize_sentence(sentence)` that

- **Input:** A string `sentence` that has been processed by the previous functions.
- **Output:** A list of strings, where each string is a lemmatized token in the sentence.

**Note:**

- You can use the `spaCy` module to tokenize the sentence and remove punctuation and spaces. Below is an example demonstrating how to implement this functionality:

  ```python
  import spacy

  nlp = spacy.load("en_core_web_sm") # Load the English language model
  
  sentence = "This is a sentence."
  doc = nlp(sentence) # Process the input sentence

  # Extract lemmatized tokens, excluding punctuation and spaces
  tokens = [
    token.lemma_ 
    for token in doc 
    if not token.is_punct and not token.is_space
  ]
  print(tokens)
  ```
  The printed output will be 
  ```python
  ['this', 'be', 'a', 'sentence']
  ```


In [128]:
import spacy

nlp = spacy.load("en_core_web_sm")

def tokenize_sentence(sentence):
    """
    Tokenizes sentence into lemmatized words
    IN: sentence, str, sentence to tokenize
    OUT: tokenized_sentence, list[str], tokenized sentences
    """
    # use spaCy to tokenize the sentence into lemmatized words
    """__Your_Code_Here__"""
    return tokenized_sentence

#### 2.3 (6 pts): Remove stopwords.

Write a function `remove_stopwords(tokens)` that

- **Input:** A list of strings `tokens` that have been processed by the previous functions.
- **Output:** A filtered list of strings that does not contain stopwords.

**Note:**

- You can use the `spaCy` module to remove the stopwords. Below is an example demonstrating how to implement this functionality, including adding an additional stopword "blah":

  ```python
  import spacy

  nlp = spacy.load("en_core_web_sm") # Load the English language model

  nlp.Defaults.stop_words.add("blah") # Add an additional stopword "blah"

  tokens = ["this", "be", "a", "sentence"]

  # Filter out stopwords from the tokens list
  tokens = [token for token in tokens if not nlp.vocab[token].is_stop]
  print(tokens)
  ```
  The printed output will be 
  ```python
  ['sentence']
  ```

In [129]:
import spacy

nlp = spacy.load("en_core_web_sm")
nlp.Defaults.stop_words.add("blah")

def remove_stopwords(tokens):
    """
    Removes stopwords from tokens
    IN: tokens, list[str], tokens to remove stopwords from
    OUT: tokens, list[str], tokens with stopwords removed
    """
    # use spaCy to remove stopwords
    """__Your_Code_Here__"""
    return tokens

### Task 3 (5 pts): Save the cleaned text

#### 3.1 (5 pts): Save the processed text as a csv file.

Write a function `save_as_csv(process_text, file_path)` that

- **Input 1:** A list of lists of strings `processed_text`, where each inner list contains the processed tokens of a sentence in a lecture transcript.
- **Input 2:** A string `file_path` that specifies the path to save the CSV file.
- **Write** the processed text to a CSV file at the specified `file_path`.

In [13]:
import csv

def save_as_csv(process_text, file_path):
    """
    Saves processed text to a CSV file
    IN: process_text, list[list[str]], processed text of a lecture transcript to save to a CSV file
        file_path, str, path to the CSV file
    OUT: None
    """
    # write the processed text to a CSV file
    with open(file_path,'w') as file:
        for sentence in process_text:
            temp = " ".join(sentence)
            file.write(temp)



### Task 4 (45 pts): Word Frequency Analysis

#### 4.1 (10 pts): Compute the cumulative word frequency.

Write a function `update_word_frequency(processed_text, cumulated_frequency)` that

- **Input 1:** A list of lists of strings `processed_text`, where each inner list contains the processed tokens of a sentence in a lecture transcript.
- **Input 2:** A dictionary `cumulated_frequency` that stores the cumulated word frequency up to now. By default, `cumulated_frequency` is an empty dictionary.

In [131]:
from collections import Counter # if you don't know Counter, ignore this import. Recall Counter is a subclass of dict

def update_word_frequency(processed_text, cumulated_frequency=None):
    """
    Updates the cumulated word frequency with the frequency in the processed text
    IN: processed_text, list[list[str]], processed text to count word frequency
        cumulated_frequency, dict{str: int}, cumulated word frequency
    OUT: cumulated_frequency, dict{str: int}, cumulated word frequency
    """
    # if cumulated_frequency is not provided, initialize it
    """__Your_Code_Here__"""

    # update the cumulated frequency with the frequency in each sentence
    """__Your_Code_Here__"""

    return cumulated_frequency

#### 4.2 (5 pts): Plot the word frequency as a histogram.

Write a function `plot_word_frequency_dist(cumulated_frequency)` that

- **Input:** A dictionary `cumulated_frequency` that stores the cumulated word frequency.
- Plot the word frequency as a histogram.

Note:

- This function plots a histogram where the x-axis represents the word frequency, and the y-axis represents the number of words that appear with the corresponding frequency.
- You may use the `matplotlib` module to create the histogram. If you haven't installed it yet, you can do so by running the following command in your terminal:
  ```bash
  pip install matplotlib
  ```
- To plot a histogram, you may use the plt.hist() function, which takes a list of values as input.

  ```python
  import matplotlib.pyplot as plt
  values = [1, 1, 1, 2, 3, 3, 3, 3, 3, 4, 4, 5]
  plt.hist(values)
  plt.show()
  ```

In [132]:
import matplotlib.pyplot as plt

def plot_word_frequency_dist(cumulated_frequency):
    """
    Plots word frequency
    IN: cumulated_frequency, dict{str: int}, cumulated word frequency
    OUT: None
    """
    # plot the word frequency distribution
    """__Your_Code_Here__"""
    

    plt.xlabel("Word Frequency")
    plt.ylabel("Num of Words")
    plt.title("Word Frequency Distribution")
    plt.show()

#### 4.3 (5 pts): Sort the words by frequency.

Write a function `sort_words_by_frequency(cumulated_frequency)` that

- **Input:** A dictionary `cumulated_frequency` that stores the cumulated word frequency.
- **Output:** A list of tuples, where each tuple contains a word and its frequency, sorted by frequency in non-ascending order. If two words have the same frequency, they are sorted in alphabetical non-descending order.

In [133]:
def sort_words_by_frequency(cumulated_frequency):
    """
    Sorts words by frequency
    IN: cumulated_frequency, dict{str: int}, cumulated word frequency
    OUT: sorted_word_frequencies, list[tuple(str, int)], sorted words by frequency
    """
    # sort the words by frequency, breaking ties by word alphabetical order
    """__Your_Code_Here__"""
    return sorted_words

#### 4.4 (10 pts): Bucket the words by frequency using mean and standard deviation.

#### 4.4.1 (3 pts): Find the mean and standard deviation of the word frequency.

Write a function `find_mean_and_std(sorted_word_frequencies)` that

- **Input:** A sorted list of tuples `sorted_word_frequencies` consisting of words and their frequencies.
- **Output:** A tuple `(mean, std)` where `mean` is the average of the word frequencies and `std` is the standard deviation of the word frequencies.

#### Formulas:
- Mean: 
  $$
  \bar{f} = \frac{\sum_{i=1}^{n} f_i}{n}
  $$
  where $f_i$ is the frequency of the $i$-th word and $n$ is the number of words.

- Standard Deviation: 
  $$
  \sigma = \sqrt{\frac{\sum_{i=1}^{n} (f_i - \bar{f})^2}{n}}
  $$
  where $f_i$ is the frequency of the $i$-th word, $\bar{f}$ is the mean of the word frequency, and $n$ is the number of words.


In [134]:
def find_mean_and_std(sorted_word_frequencies):
    """
    Finds mean and standard deviation of word frequency
    IN: sorted_word_frequencies, list[tuple(str, int)], sorted words by frequency
    OUT: mean, float, mean of word frequency
         std, float, standard deviation of word frequency
    """
    # access all the frequencies
    """__Your_Code_Here__"""

    # calculate the mean
    """__Your_Code_Here__"""

    # calculate the standard deviation
    """__Your_Code_Here__"""
    
    return mean, std

##### 4.4.2 (3 pts): Bucket a value by the mean and standard deviation.

Write a function `bucket_value_by_mean_std(value, mean, std)` that

- **Input 1:** A number `value` representing the frequency of a word.
- **Input 2:** A float `mean` representing the mean of the word frequency.
- **Input 3:** A float `std` representing the standard deviation of the word frequency.
- **Output:** An integer representing the bucket index of the word frequency.

**Formula:**
The bucket index can be calculated using the formula:
$$
b = \left\lfloor \frac{f - \bar{f}}{\sigma} \right\rfloor
$$
where:
- $f$ is the frequency of the word,
- $\bar{f}$ is the mean of the word frequency,
- $\sigma$ is the standard deviation of the word frequency,
- $\lfloor \cdot \rfloor$ is the floor function.

In [135]:
import math

def bucket_value_by_mean_std(value, mean, std):
    """
    Buckets value by mean and standard deviation
    IN: value, int | float, value from a distribution to bucket
        mean, float, mean of the distribution
        std, float, standard deviation of the distribution
    OUT: bucket_idx, int, bucket index
    """
    # calculate the bucket index
    """__Your_Code_Here__"""
    
    return bucket_idx

##### 4.4.3 (4 pts): Find the bucket index of each word.

Write a function `find_bucket_index_of_words(sorted_word_frequencies)` that

- **Input:** A sorted list of tuples `sorted_word_frequencies` consisting of words and their frequencies.
- **Output:** A list of dictionaries, where each dictionary contains three key-value pairs:
  - `word`: the word from the input list.
  - `frequency`: the frequency of the word.
  - `bucket`: the bucket index of the word frequency.

In [136]:
def find_bucket_index_of_words(sorted_word_frequencies):
    """
    Finds bucket index of words by mean and standard deviation
    IN: sorted_word_frequencies, list[tuple(str, int)], sorted words by frequency
    OUT: sorted_word_info, list[dict{"word": str, "frequency": int, "bucket": int}], words with frequency and bucket index
    """
    # find the mean and standard deviation of the word frequencies
    """__Your_Code_Here__"""

    # collect the words with their frequency and bucket index
    sorted_word_info = []
    for word, freq in sorted_word_frequencies:
        """__Your_Code_Here__"""

    return sorted_word_info


#### 4.5 (5 pts): Save the word information as a JSON file.

Write a function `save_as_json(sorted word_info, file_path)` that

- **Input 1:** A sorted list of dictionaries `sorted_word_info` that contains frequency and bucket information of each word.
- **Input 2:** A string `file_path` that specifies the path to save the JSON file.
- **Write** the word information as a JSON file.

In [137]:
import json

def save_as_json(sorted_word_info, file_path):
    """
    Saves word info to a JSON file
    IN: sorted_word_info, list[dict{"word": str, "frequency": int, "bucket": int}], words with frequency and bucket index
        file_path, str, path to the JSON file
    OUT: None
    """
    # write the word info to a JSON file
    """__Your_Code_Here__"""

#### 4.6 (10 pts): Plot stacked word frequency bar chart.

Write a function `plot_stacked_word_frequency(word_frequencies_list, sorted_words)` that

- **Input 1:** A list of dictionaries `word_frequencies_list`. Each dictionary contains the cumulated word frequency up to a certain lecture transcript. 
  
  The first dictionary contains the word frequency of the first lecture transcript, the second dictionary contains the cumulated word frequency of the first and second lecture transcripts, and so on.
- **Input 2:** A list of tuples `sorted_words` that contains the words sorted by frequency.
- **Plot** the word frequency as a stacked bar chart.

Note:

- A stacked bar chart uses bars to show comparisons between categories of data, with the ability to break down and compare parts of a whole. In this task:
  - The x-axis represents the words.
  - The y-axis represents the word frequency.
  - Bars are sorted using the order of `sorted_words`.
- You may use the `matplotlib` module to plot the stacked bar chart. The following code snippet demonstrates how to create a stacked bar chart using the `plt.bar()` function.

  ```python
  import matplotlib.pyplot as plt
  
  cmap = plt.get_cmap("viridis_r")  # use colormap for coloring the bars
  
  words = ["apple", "banana", "cherry"]  # list all words used in x-axis
  frequencies_list = [  # list of cumulated word frequencies
      [1, 3, 0],  # frequencies in document 1
      [3, 3, 3],  # frequencies in document 1 and 2
      [6, 5, 3],  # frequencies in document 1, 2, and 3
  ]
  
  for i, frequencies in reversed(list(enumerate(frequencies_list))):  # plot in reverse order, so the earlier one is on top
      plt.bar(
          words, # x-axis, words
          frequencies, # y-axis, cumulated frequencies
          color=cmap(i / (len(frequencies_list) - 1)), # color the bars
          label=f"Up to Document {i+1}", # label the part of the bar
      )
  
  plt.xlabel("Words")
  plt.xticks(rotation=90) # rotate the x-axis labels for better readability
  plt.ylabel("Frequency")
  plt.title("Word Frequency in Documents")
  plt.legend()
  
  plt.show()
  ```

In [138]:
import matplotlib.pyplot as plt

def plot_stacked_word_frequency(word_frequencies_list, sorted_words):
    """
    Plots stacked word frequency
    IN: word_frequencies_list, list[dict{str: int}], word frequencies to plot
        sorted_words, list[tuple(str, int)], sorted words by frequency
    OUT: None
    """
    # set up color map
    cmp = plt.get_cmap("viridis_r")

    # get words and frequencies
    """__Your_Code_Here__"""

    # plot the stacked word frequency
    """__Your_Code_Here__"""

    plt.xlabel("Words")
    plt.xticks(rotation=90)
    plt.ylabel("Frequency")
    plt.title("Stacked Word Frequency")
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5)) # move the legend to the right to avoid overlapping with the bars
    plt.show()

### Call of functions for testing purposes

In [ ]:
if __name__ == "__main__":
    # this block of code is executed when directly running the script, not when importing the script as a module
    """
    Feel free to modify the code inside this if block to test your functions
    """
    word_frequencies_list = []
    cumulated_frequency = None
    for i in range(1, 13):
        fileName = f'./data/transcript {i}.pdf'
        print(f"Processing {fileName}")

        # extract text from the PDF file
        text = extract_pdf_text(fileName)
        text = remove_headers(text)
        text = remove_parentheses(text)
        text = remove_speakers(text)

        # natural language preprocessing
        sentences = split_into_sentences(text)
        processed_text = []
        for sentence in sentences:
            tokens = tokenize_sentence(sentence)
            tokens = remove_stopwords(tokens)
            processed_text.append(tokens)

        # update count of word frequency
        cumulated_frequency = update_word_frequency(processed_text, cumulated_frequency)
        word_frequencies_list.append(cumulated_frequency.copy())

    # plot the word final cumulative frequency distribution
    plot_word_frequency_dist(word_frequencies_list[-1])

    # sort words by final cumulative frequency
    sorted_words = sort_words_by_frequency(word_frequencies_list[-1])

    # find bucket index of all words
    sorted_word_info = find_bucket_index_of_words(sorted_words)

    # take words whose bucket index >= 5 (i.e. frequency >= mean + 5 * std)
    filtered_words = [(info['word'], info['frequency']) for info in sorted_word_info if info['bucket'] >= 5]

    # plot stacked word frequency
    plot_stacked_word_frequency(word_frequencies_list, filtered_words)
    